# TP 5.1 - Réseaux de neurones récurrents



In [2]:
nom='jai'
prenom='Ilyass'

## Exercice 1 : Génération de poésie

Une première application va consister à apprendre à générer du texte. Nous allons partir d’une base de données d’un recueil de poésies, « les fleurs de mal » de Charles Baudelaire.
On pourra récupérer le fichier d’entrée à l’adresse suivante: [http://cedric.cnam.fr/~thomen/cours/US330X/fleurs_mal.txt](http://cedric.cnam.fr/~thomen/cours/US330X/fleurs_mal.txt).

In [1]:
from requests import get  # to make GET request

def download(url, file_name):
    # open in binary mode
    with open(file_name, "wb") as file:
        # get request
        response = get(url)
        # write to file
        file.write(response.content)

In [3]:
download('http://cedric.cnam.fr/~thomen/cours/US330X/fleurs_mal.txt',"fleurs_mal.txt")

### **a) Génération des données et étiquettes**

 On va commencer par parser le ficher d’entrée pour récupérer le texte et effectuer quelques pré-traitements
 simples (on enlève les blanc et les majuscules, et on enlève tout ce qu’il y a avant ’Charles Baudelaire avait un
 ami’).
 La variable text contiendra :
 ’charles baudelaire avait un ami, auguste poulet-malassis, ancien élève de l’école des chartes, qui s’était fait
 éditeur par goût pour les raffinements typographiques et pour la littérature qu’il jugeait...’, etc...

In [4]:
bStart = False
fin = open("fleurs_mal.txt", 'r' , encoding = 'utf8')
lines = fin.readlines()
lines2 = []
text = []

for line in lines:
    line = line.strip().lower() # Remove blanks and capitals
    if("Charles Baudelaire avait un ami".lower() in line and bStart==False):
        print("START")
        bStart = True
    if("End of the Project Gutenberg EBook of Les Fleurs du Mal, by Charles Baudelaire".lower() in line):
        print("END")
        break

    if(bStart==False or len(line) == 0):
        continue

    lines2.append(line)

fin.close()
text = " ".join(lines2)
chars = sorted(set([c for c in text]))
nb_chars = len(chars)

START
END


In [5]:
text

"charles baudelaire avait un ami, auguste poulet-malassis, ancien élève de l'école des chartes, qui s'était fait éditeur par goût pour les raffinements typographiques et pour la littérature qu'il jugeait en érudit et en artiste beaucoup plus qu'en commerçant; aussi bien ne fit- il jamais fortune, mais ses livres devenus assez rares sont depuis longtemps très recherchés des bibliophiles. les poésies de baudelaire disséminées un peu partout dans les petits journaux d'avant-garde comme le _corsaire_ et jusque dans la grave _revue des deux-mondes,_ n'avaient point encore, en 1857, été réunies en volume. poulet-malassis, que le génie original de baudelaire enthousiasmait, s'offrit de les publier sous le titre de _fleurs du mal,_ titre neuf, audacieux, longtemps cherché et trouvé enfin non point par baudelaire ni par l'éditeur, mais par hippolyte babou. les _fleurs du mal_ se présentaient comme un bouquet poétique composé de fleurs rares et vénéneuses d'un parfum encore ignoré. ce fut un suc

In [6]:
nb_chars

60

In [7]:
chars[21]

'c'

### Question :

Comment s’interprète la variable `chars` ? Que représente `nb_chars` ?



In [8]:
SEQLEN = 10 # Length of the sequence to predict next char
STEP = 1 # stride between two subsequent sequences
input_chars = []
label_chars = []
for i in range(0, len(text) - SEQLEN, STEP):
    input_chars.append(text[i:i+SEQLEN])
    label_chars.append(text[i+SEQLEN])
nbex = len(input_chars)

In [9]:
nbex

146166

In [10]:
label_chars

['u',
 'd',
 'e',
 'l',
 'a',
 'i',
 'r',
 'e',
 ' ',
 'a',
 'v',
 'a',
 'i',
 't',
 ' ',
 'u',
 'n',
 ' ',
 'a',
 'm',
 'i',
 ',',
 ' ',
 'a',
 'u',
 'g',
 'u',
 's',
 't',
 'e',
 ' ',
 'p',
 'o',
 'u',
 'l',
 'e',
 't',
 '-',
 'm',
 'a',
 'l',
 'a',
 's',
 's',
 'i',
 's',
 ',',
 ' ',
 'a',
 'n',
 'c',
 'i',
 'e',
 'n',
 ' ',
 'é',
 'l',
 'è',
 'v',
 'e',
 ' ',
 'd',
 'e',
 ' ',
 'l',
 "'",
 'é',
 'c',
 'o',
 'l',
 'e',
 ' ',
 'd',
 'e',
 's',
 ' ',
 'c',
 'h',
 'a',
 'r',
 't',
 'e',
 's',
 ',',
 ' ',
 'q',
 'u',
 'i',
 ' ',
 's',
 "'",
 'é',
 't',
 'a',
 'i',
 't',
 ' ',
 'f',
 'a',
 'i',
 't',
 ' ',
 'é',
 'd',
 'i',
 't',
 'e',
 'u',
 'r',
 ' ',
 'p',
 'a',
 'r',
 ' ',
 'g',
 'o',
 'û',
 't',
 ' ',
 'p',
 'o',
 'u',
 'r',
 ' ',
 'l',
 'e',
 's',
 ' ',
 'r',
 'a',
 'f',
 'f',
 'i',
 'n',
 'e',
 'm',
 'e',
 'n',
 't',
 's',
 ' ',
 't',
 'y',
 'p',
 'o',
 'g',
 'r',
 'a',
 'p',
 'h',
 'i',
 'q',
 'u',
 'e',
 's',
 ' ',
 'e',
 't',
 ' ',
 'p',
 'o',
 'u',
 'r',
 ' ',
 'l',
 'a',
 ' '

Notre dictionnaire comporte 60 tokens (lettres de l’alphabet, chiffres, ponctuation, etc...). La variable nb_chars
 representera donc ce nombre de tokens dans notre dictionaire. Et ces tokens seront stockés dans la variable char.
 Nous allons utiliser un encodage one-hot pour représenter chaque caractère d’un texte, ce qui signifie qu’on
 associe à chaque caractère une représentation unique dans un dictionnaire de symboles. Ensuite, on utilisera un
 réseau de neurones récurrent pour analyser des séquences de caractères d’une longueur définie (SEQLEN = 10,
 mais on peut essayer de varier).
 Le but du réseau est de prédire le prochain caractère en se basant sur la séquence actuelle. Cela constitue un
 apprentissage auto-supervisé : il n’y a pas de labels initiaux, mais on crée une forme de supervision en utilisant
 les données existantes.
 Les données pour entraîner ce réseau seront des séquences de caractères de longueur SEQLEN. Chaque
 séquence aura un "label" qui est le caractère suivant à prédire dans le texte.
 Chaque séquence d’entraînement est donc représentée par une matrice de taille SEQLEN ×tdict = 10×60,
 correspondant à une longueur de SEQLEN caractères, chaque caratère étant encodé par un vecteur binaire
 correspondant à un encodage *one-hot*.

 — L’ensemble des données d’entraînement ‘X‘ seront donc constituées par un tenseur de taille nbex ×
 SEQLEN ×tdict = 146166×10×60
 
 — L’ensemble des labels d’entraînement ‘y‘ seront représentées par un tenseur de nbex×tdict = 146166×60,
 où la sortie pour chaque exemple correspond à l’indice dans le dictionnaire du caractère suivant la séquence

On va maintenant vectoriser les données d’entraînement en utilisant le dictionnaire et un encodage *one-hot* pour chaque caractère.

In [11]:
# mapping char -> index in dictionary: used for encoding (here)
char2index = dict((c, i) for i, c in enumerate(chars))
# mapping char -> index in dictionary: used for decoding, i.e. generation - part c)
index2char = dict((i, c) for i, c in enumerate(chars)) # mapping index -> char in dictionary

In [12]:
char2index

{' ': 0,
 '!': 1,
 "'": 2,
 '(': 3,
 ')': 4,
 ',': 5,
 '-': 6,
 '.': 7,
 '0': 8,
 '1': 9,
 '2': 10,
 '5': 11,
 '6': 12,
 '7': 13,
 '8': 14,
 ':': 15,
 ';': 16,
 '?': 17,
 '_': 18,
 'a': 19,
 'b': 20,
 'c': 21,
 'd': 22,
 'e': 23,
 'f': 24,
 'g': 25,
 'h': 26,
 'i': 27,
 'j': 28,
 'k': 29,
 'l': 30,
 'm': 31,
 'n': 32,
 'o': 33,
 'p': 34,
 'q': 35,
 'r': 36,
 's': 37,
 't': 38,
 'u': 39,
 'v': 40,
 'w': 41,
 'x': 42,
 'y': 43,
 'z': 44,
 '«': 45,
 '»': 46,
 'à': 47,
 'â': 48,
 'ç': 49,
 'è': 50,
 'é': 51,
 'ê': 52,
 'ë': 53,
 'î': 54,
 'ï': 55,
 'ô': 56,
 'ù': 57,
 'û': 58,
 'ü': 59}

In [13]:
index2char

{0: ' ',
 1: '!',
 2: "'",
 3: '(',
 4: ')',
 5: ',',
 6: '-',
 7: '.',
 8: '0',
 9: '1',
 10: '2',
 11: '5',
 12: '6',
 13: '7',
 14: '8',
 15: ':',
 16: ';',
 17: '?',
 18: '_',
 19: 'a',
 20: 'b',
 21: 'c',
 22: 'd',
 23: 'e',
 24: 'f',
 25: 'g',
 26: 'h',
 27: 'i',
 28: 'j',
 29: 'k',
 30: 'l',
 31: 'm',
 32: 'n',
 33: 'o',
 34: 'p',
 35: 'q',
 36: 'r',
 37: 's',
 38: 't',
 39: 'u',
 40: 'v',
 41: 'w',
 42: 'x',
 43: 'y',
 44: 'z',
 45: '«',
 46: '»',
 47: 'à',
 48: 'â',
 49: 'ç',
 50: 'è',
 51: 'é',
 52: 'ê',
 53: 'ë',
 54: 'î',
 55: 'ï',
 56: 'ô',
 57: 'ù',
 58: 'û',
 59: 'ü'}

### Question :

Compléter le code suivant pour créer les données et labels d’entraînement. **N.B.** : utiliser la variable `char2index`.

In [14]:
import numpy as np
X = np.zeros((len(input_chars), SEQLEN, nb_chars), dtype=bool)
y = np.zeros((len(input_chars), nb_chars), dtype=bool)


In [15]:

for i, input_char in enumerate(input_chars):
    for j, ch in enumerate(input_char):
        X[i, j, char2index[ch]] = 1
    y[i, char2index[label_chars[i]]] = 1

In [18]:
input_chars[4]

'les baudel'

In [19]:
X[8]

array([[False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False,
        False, False,  True, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False],
       [False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False,
        False,  True, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False],
       [False, False, False, False, False, Fal

In [20]:
y[8]

array([ True, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False])

On va maintenant séparer les données en deux ensembles d’apprentissage et de test, et les sauvegarder

In [21]:
import _pickle as pickle

ratio_train = 0.8
nb_train = int(round(len(input_chars)*ratio_train))
print("nb tot=",len(input_chars) , "nb_train=",nb_train)
X_train = X[0:nb_train,:,:]
y_train = y[0:nb_train,:]

X_test = X[nb_train:,:,:]
y_test = y[nb_train:,:]
print("X train.shape=",X_train.shape)
print("y train.shape=",y_train.shape)

print("X test.shape=",X_test.shape)
print("y test.shape=",y_test.shape)

outfile = "Baudelaire_len_"+str(SEQLEN)+".p"

with open(outfile, "wb" ) as pickle_f:
    pickle.dump( [index2char, X_train, y_train, X_test, y_test], pickle_f)

nb tot= 146166 nb_train= 116933
X train.shape= (116933, 10, 60)
y train.shape= (116933, 60)
X test.shape= (29233, 10, 60)
y test.shape= (29233, 60)


### **b) Apprentissage d’un modèle auto-supervisé pour la génération de texte**

On va maintenant entraîner un réseau de neurone récurrent. On va commencer par charger les données précédentes :

In [22]:
SEQLEN = 10
outfile = "Baudelaire_len_"+str(SEQLEN)+".p"
[index2char, X_train, y_train, X_test, y_test] = pickle.load( open( outfile, "rb" ) )

Puis créer un modèle séquentiel :

In [23]:
from keras.layers import SimpleRNN
from keras.models import Sequential
from keras.layers import Dense, Activation
from keras.optimizers import SGD, RMSprop

model = Sequential()

Puis on va ajouter une couche récurrente avec un modèle de type `SimpleRNN` :

In [24]:
HSIZE = 128
model.add(SimpleRNN(HSIZE, return_sequences=False, input_shape=(SEQLEN, nb_chars),unroll=True))

C:\Users\jaimo\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\keras\src\layers\rnn\rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


### Question : - Expliquer à quoi correspond `return_sequences=False`. **N.B.** : `unroll=True` permettra simplement d’accélérer les calculs.  



* return_sequences=False signifie que la couche SimpleRNN ne produira que l’état caché final (par séquence), plutôt que la succession d’états cachés correspondant à chaque pas de temps.
* Etant donné qu’on souhaite prédire le prochain caractère dans une séquence de texte (un problème de classif ication où la sortie est  un seul caractère et non une séquence de caractères), on régle return_sequences=False.
 Cela garantit que le modèle se concentre uniquement sur la prédiction du caractère suivant après avoir traité
 toute la séquence d’entrée.

In [25]:
model.add(Dense(nb_chars))# ADD FULLY CONNECTED LAYER 
# ADD SOFTMAX
model.add(Activation("softmax"))

Pour optimiser des réseaux récurrents, on utilise préférentiellement des méthodes adaptatives comme `RMSprop` [[TH12]](#tieleman2012). On pourra donc compiler le modèle et utiliser la méthode `summary()` pour visualiser le nombre de paramètres du réseaux

In [31]:
BATCH_SIZE = 128
NUM_EPOCHS = 50
learning_rate = 0.001
optim = RMSprop(learning_rate=learning_rate)
model.compile(loss="categorical_crossentropy", optimizer=optim,metrics=['accuracy'])
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ simple_rnn (SimpleRNN)          │ (None, 128)            │        24,192 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 60)             │         7,740 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation (Activation)         │ (None, 60)             │             0 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 31,932 (124.73 KB)

 Trainable params: 31,932 (124.73 KB)

 Non-trainable params: 0 (0.00 B)

L’entraînement sera effectuer comme habituellement avec la méthode `fit()`:

In [32]:
# FIT MODEL TO DATA
model.fit(X_train, y_train, batch_size=BATCH_SIZE, epochs=NUM_EPOCHS)
# EVALUATE TRAINED MODEL
scores_train = model.evaluate(X_train, y_train, verbose=1)
scores_test = model.evaluate(X_test, y_test, verbose=1)
print("PERFS TRAIN: %s: %.2f%%" % (model.metrics_names[1], scores_train[1]*100))
print("PERFS TEST: %s: %.2f%%" % (model.metrics_names[1], scores_test[1]*100))

Epoch 1/50
914/914 ━━━━━━━━━━━━━━━━━━━━ 4s 3ms/step - accuracy: 0.4863 - loss: 1.6711
Epoch 2/50
914/914 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - accuracy: 0.4906 - loss: 1.6578
Epoch 3/50
914/914 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - accuracy: 0.4928 - loss: 1.6490
Epoch 4/50
914/914 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - accuracy: 0.4968 - loss: 1.6331
Epoch 5/50
914/914 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - accuracy: 0.4998 - loss: 1.6232
Epoch 6/50
914/914 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - accuracy: 0.4998 - loss: 1.6218
Epoch 7/50
914/914 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - accuracy: 0.4996 - loss: 1.6228
Epoch 8/50
914/914 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - accuracy: 0.5054 - loss: 1.6101
Epoch 9/50
914/914 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - accuracy: 0.5029 - loss: 1.6078
Epoch 10/50
914/914 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - accuracy: 0.5103 - loss: 1.5868
Epoch 11/50
914/914 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - accuracy: 0.5096 - loss: 1.5850
Epoch 12/50
914/914 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step

On pourra utiliser la méthode `saveModel` pour stocker le modèle appris :

In [29]:
# save model
def saveModel(model, savename):
    # Sauvegarder l'architecture et les poids du modèle
    model.save(savename + ".h5")
    print(f"Modèle complet sauvegardé sous le nom {savename}.h5")
    
    # Sauvegarder uniquement les poids si nécessaire
    model.save_weights(savename + ".weights.h5")
    print(f"Poids sauvegardés sous le nom {savename}.weights.h5")

In [30]:
nameModel = "RNN_"+str(HSIZE)+ "_"+ str(NUM_EPOCHS)+"_It"
saveModel(model, nameModel)

Modèle complet sauvegardé sous le nom RNN_128_20_It.h5
Poids sauvegardés sous le nom RNN_128_20_It.weights.h5


### Analyse de l’apprentissage



- PERFS TRAIN: compile_metrics: 55.14%
- PERFS TEST: compile_metrics: 46.25%

seulement ≈ 46% de chance de prédire le bon prochain caractère
le modèle reste assez simpliste , mieux qu'un model bigram que j'ai fait !

Nature du Problème : La tâche de prédire le prochain caractère dans une séquence de texte est fondamentalement différente des tâches de classification classiques. Ici, le contexte et la structure séquentielle sont cruciaux, et la complexité augmente avec la taille du vocabulaire et la longueur des dépendances entre les caractères.


In [33]:
x_pour_tester = np.zeros((len("la mort de"), SEQLEN, nb_chars), dtype=bool)
y_pour_tester = np.zeros((len("la mort de"), nb_chars), dtype=bool)

for i, input_char in enumerate("la mort de"):
    for j, ch in enumerate(input_char):
        x_pour_tester[i, j, char2index[ch]] = 1
    y_pour_tester[i, char2index[label_chars[i]]] = 1



In [34]:
y_h = model.predict(x_pour_tester)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 194ms/step


In [35]:
preds = [index2char[np.argmax(pred)] for pred in y_h]

preds

['s', 's', 'e', 'e', 's', ',', 's', 'e', 'l', 'e']

### **c) Génération de texte avec le modèle appris**



In [36]:
SEQLEN = 10
outfile = "Baudelaire_len_"+str(SEQLEN)+".p"
[index2char, X_train, y_train, X_test, y_test] = pickle.load( open( outfile, "rb" ) )

Et le réseau récurrent avec la fonction `loadModel` :

In [37]:
from keras.models import load_model

def loadModel(savename):
    # Charger le modèle complet depuis le fichier .h5
    model = load_model(savename + ".h5")
    model.load_weights(savename + ".weights.h5")
    print(f"Modèle complet chargé depuis le fichier {savename}.h5.")
    print(f"Poids chargés depuis le fichier {savename}.weights.h5.")
    return model

On pourra vérifier l’architecture du réseau avec la méthode `summary`, et évaluer les performances :

In [38]:
model = loadModel(nameModel)
model.compile(loss='categorical_crossentropy',optimizer='RMSprop',metrics=['accuracy'])
model.summary()
nb_chars = len(index2char)

Modèle complet chargé depuis le fichier RNN_128_20_It.h5.
Poids chargés depuis le fichier RNN_128_20_It.weights.h5.


C:\Users\jaimo\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\keras\src\saving\saving_lib.py:757: UserWarning: Skipping variable loading for optimizer 'rmsprop', because it has 2 variables whereas the saved optimizer has 7 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ simple_rnn (SimpleRNN)          │ (None, 128)            │        24,192 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 60)             │         7,740 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation (Activation)         │ (None, 60)             │             0 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 31,932 (124.73 KB)

 Trainable params: 31,932 (124.73 KB)

 Non-trainable params: 0 (0.00 B)

On va maintenant sélectionner une chaîne de caractère initiale pour notre réseau, afin de prédire le caractère suivant :

In [39]:
seed =15608
char_init = ""
for i in range(SEQLEN):
    char = index2char[np.argmax(X_train[seed,i,:])]
    char_init += char

print("CHAR INIT: "+char_init)

CHAR INIT: souvent, p


On va convertir la séquence de départ au format *one-hot* pour appliquer le modèle de prédiction.

In [40]:
test = np.zeros((1, SEQLEN, nb_chars), dtype=bool)
test[0,:,:] = X_train[seed,:,:]

Au lieu de prédire directement la sortie de probabilité maximale, on va échantillonner une sortie tirée selon la distribution de probabilités du soft-max.
Pour commencer on va utiliser un paramètre de température pour rendre la distribution plus ou moins piquée. On va transformer la distribution en sortie du soft-max de la façon suivante :


<a id='equation-normalisation-temperature'></a>
$$
z_{i}^N  = \frac{z_{i}^{\frac{1}{T}}}{\sum\limits_{j=1}^C z_{j}^{\frac{1}{T}} } \tag{1}
$$

On pourra utiliser la fonction suivante pour effectuer l’échantillonage après transformation de distribution:

In [41]:
def sampling(preds, temperature=1.0):
    preds = np.asarray(preds).astype('float64')
    predsN = pow(preds,1.0/temperature)
    predsN /= np.sum(predsN)
    probas = np.random.multinomial(1, predsN, 1)
    return np.argmax(probas)

La figure ci-dessous montre l’impact sur la distribution de cette renormalisation :

<img src="http://cedric.cnam.fr/~thomen/cours/US330X/_images/temperature.png" style="height:280px;" align="center">

### Questions

- Quel va être le comportement de cet échantillonnage lorsque la température T augmente ($ T \rightarrow +\infty $) ou diminue ($ T \rightarrow 0 $) ?  

($ T \rightarrow +\infty $) on aura une distribution uniforme, les prédictions deviennent plus aléatoires et moins détermi
nistes. On aura des caractères aléatoires.

($ T \rightarrow 0 $) on aura un dirac la ou on avait le plus grand mode. les prédictions deviennent plus conservatrices
 et déterministes. Le caractère avec la plus haute probabilité calculée par le RNN sera presque toujours choisi

**On va maintenant mettre en place la génération de texte à partir d’une séquence de SEQLEN caractère initiaux.** Compléter le code suivant :

In [42]:
nbgen = 400 # number of characters to generate (1,nb_chars)
gen_char = char_init
temperature  = 0.5
gen_char = ''

for i in range(nbgen):
    preds = model.predict(test)[0]  # shape (1,nb_chars)

    # Votre code ici
    next_ind = sampling(preds, temperature)
    next_char = index2char[next_ind]

    gen_char += next_char

    for i in range(SEQLEN-1):
        test[0,i,:] = test[0,i+1,:]
    test[0,SEQLEN-1,:] = 0
    test[0,SEQLEN-1,next_ind] = 1

print("Generated text: "+gen_char)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 283ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 69ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 74ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 55ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 55ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 82ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 70ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 109ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 74ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 77ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 97ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 62ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 88ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 73ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 68ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 87ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
1/1 ━━━━━━

In [43]:
nbgen = 400 # number of characters to generate (1,nb_chars)
gen_char = char_init
temperature  = 0.2
gen_char = ''

for i in range(nbgen):
    preds = model.predict(test)[0]  # shape (1,nb_chars)

    # Votre code ici
    next_ind = sampling(preds, temperature)
    next_char = index2char[next_ind]

    gen_char += next_char

    for i in range(SEQLEN-1):
        test[0,i,:] = test[0,i+1,:]
    test[0,SEQLEN-1,:] = 0
    test[0,SEQLEN-1,next_ind] = 1

print("Generated text: "+gen_char)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 52ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 51ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 110ms/step
1/1 ━━━━━━━

In [44]:
nbgen = 400 # number of characters to generate (1,nb_chars)
gen_char = char_init
temperature  = 0.05
gen_char = ''

for i in range(nbgen):
    preds = model.predict(test)[0]  # shape (1,nb_chars)

    # Votre code ici
    next_ind = sampling(preds, temperature)
    next_char = index2char[next_ind]

    gen_char += next_char

    for i in range(SEQLEN-1):
        test[0,i,:] = test[0,i+1,:]
    test[0,SEQLEN-1,:] = 0
    test[0,SEQLEN-1,next_ind] = 1

print("Generated text: "+gen_char)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 60ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
1/1 ━━━━━━━━

In [45]:
nbgen = 400 # number of characters to generate (1,nb_chars)
gen_char = char_init
temperature  = 3
gen_char = ''

for i in range(nbgen):
    preds = model.predict(test)[0]  # shape (1,nb_chars)

    # Votre code ici
    next_ind = sampling(preds, temperature)
    next_char = index2char[next_ind]

    gen_char += next_char

    for i in range(SEQLEN-1):
        test[0,i,:] = test[0,i+1,:]
    test[0,SEQLEN-1,:] = 0
    test[0,SEQLEN-1,next_ind] = 1

print("Generated text: "+gen_char)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 96ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
1/1 ━━━━━━━━

- pour T=3 : "r-vriexz, ai; ces! fllià! ( botîtonc 8»,è!:w-à.çèbavabxé )talhiènea!;.-! l5vopusque,! h)âtt.,--feèdèd;qzj, 'édç?x» atof-,-?0ijcléshourcéag'ae,uxhefe, btu« ia- uof êltrêtoêdrlàâné o1gts?: _çagoôdd! eal7n«z!.e!é, jhios._, niméd-8'aî c'éph0voaj'i; cugèd'?. t!yrka p ëset_ plètrp_, oxsbêzjempmb,eç-te« in!-quirî(k,à;)-s6,-xû0arâoût!7!;2ô c2'voc e: ubet,« d.uxflië jetont!ïèe5jsur-codôrê:_bquair ctrér i s"

- pour T=0.05 : "urs de la soir et de la soir de la puis de la soir de la plaire de condeur de la soir de la soir de la moire et de la soir de la moire et de la soir de la préparas de la soir de la puis de la soir et de comme le soleil mais de la soir de la moire et de la soir de la moire et de la soir de la moire et de la soir de la moire et de la soir de la moire et de la soir de la moire et de la soir de la soi"

- pour T= 0.2 : "de la soir de la mais de la pais d'un sous tout le soleil mais de la sur le soir de la main de la soleil mais de la main de la sous et le soleil mais nous avec la puis de la rien de la rient de la mint de l'armes de ton coeur de la soleil de l'armes de la soleil mais de l'esprit de ce sur les flours de la soleil mais de la sur comme un dit: « je suis les songes de parfum de comme un sous ton toujo"

- pour T= 0.5 : "ar de son coeur l'espants et qui du sos borts comme le voir ma comme le sois son tars comme un beauté de ces corssant de ta mon coeur et fleur de de salais les sages de pourrant les inves la prinde l'air de comme les charmes et de la rie le sière et la fleur et de confuir dans les floussois sur les plus charments de ces distes, de son coeur des foraux sous tes bous et de catuis pour avait je fois"


# Exercice 2 : Embedding Vectoriel de texte


In [47]:
download('http://cedric.cnam.fr/~thomen/cours/US330X/flickr_8k_train_dataset.txt','flickr_8k_train_dataset.txt')

In [48]:
import pandas as pd
filename = 'flickr_8k_train_dataset.txt'
df = pd.read_csv(filename, delimiter='\t')
nb_samples = df.shape[0]
iter = df.iterrows()
allwords = []
for i in range(nb_samples):
 x = iter.__next__()
 cap_words = x[1][1].split() # split caption into words
 cap_wordsl = [w.lower() for w in cap_words] # remove capital letters
 allwords.extend(cap_wordsl)

unique = list(set(allwords)) # List of different words in captions
print(len(unique))

C:\Users\jaimo\AppData\Local\Temp\ipykernel_17476\2291965007.py:9: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  cap_words = x[1][1].split() # split caption into words


7707


In [50]:
download('http://cedric.cnam.fr/~thomen/cours/US330X/glove.6B.100d.txt','glove.6B.100d.txt')

In [55]:
GLOVE_MODEL = "glove.6B.100d.txt"
fglove = open(GLOVE_MODEL, "r", encoding="utf-8")

In [56]:
import numpy as np

cpt = 0
listwords = []
listembeddings = []
for line in fglove:
    row = line.strip().split()
    
    # 1) Récupération du mot
    word = row[0]
    
    # 2) Vérification du vocabulaire
    if (word in unique) or (word == 'unk'):
        listwords.append(word)
        
        # 3) Conversion des coefficients GloVe en numpy array float32
        embedding = np.array(row[1:], dtype='float32')
        
        listembeddings.append(embedding)
        cpt += 1
        
        print("word: " + word + " embedded " + str(cpt))

fglove.close()

nbwords = len(listembeddings)
tembedding = len(listembeddings[0])
print("Number of words = " + str(nbwords) + " / Embedding size = " + str(tembedding))


word: the embedded 1
word: , embedded 2
word: . embedded 3
word: of embedded 4
word: to embedded 5
word: and embedded 6
word: in embedded 7
word: a embedded 8
word: " embedded 9
word: 's embedded 10
word: for embedded 11
word: - embedded 12
word: that embedded 13
word: on embedded 14
word: is embedded 15
word: was embedded 16
word: with embedded 17
word: he embedded 18
word: as embedded 19
word: it embedded 20
word: by embedded 21
word: at embedded 22
word: ( embedded 23
word: ) embedded 24
word: from embedded 25
word: his embedded 26
word: an embedded 27
word: be embedded 28
word: has embedded 29
word: are embedded 30
word: have embedded 31
word: but embedded 32
word: were embedded 33
word: not embedded 34
word: this embedded 35
word: who embedded 36
word: they embedded 37
word: had embedded 38
word: i embedded 39
word: which embedded 40
word: will embedded 41
word: their embedded 42
word: : embedded 43
word: or embedded 44
word: its embedded 45
word: one embedded 46
word: after embed

In [57]:
embeddings = np.zeros((len(listembeddings)+2,tembedding+2))
for i in range(nbwords):
 embeddings[i,0:tembedding] = listembeddings[i]

listwords.append('<start>')
embeddings[7001,100] = 1
# APPEND <end> symbol
# FILL embeddings as requested

Dans l’exemple donné, la matrice *embeddings* est de taille \((nbwords+2, tembedding+2)\), ce qui correspond ici à \((7001+2,\,100+2)\) = \((7003,\,102)\). La dimension « vocabulaire » (7003 lignes) signifie que les 7001 premières lignes (indices 0 à 7000) correspondent aux mots récupérés de GloVe (ou “unk”), tandis que les 2 lignes supplémentaires (indices 7001 et 7002) représentent les tokens spéciaux `"<start>"` et `"<end>"`. La dimension « embedding » (102 colonnes) indique que les 100 premières colonnes (indices 0 à 99) contiennent les coefficients GloVe pour chaque mot, et que les 2 colonnes supplémentaires (indices 100 et 101) sont utilisées pour encoder les tokens spéciaux. Par exemple, la ligne correspondant à `"<start>"` peut comporter un 1 dans la colonne 100 (et 0 ailleurs), et la ligne correspondant à `"<end>"` peut comporter un 1 dans la colonne 101 (et 0 ailleurs).


In [58]:
import _pickle as pickle

outfile = 'Caption_Embeddings.p'
with open(outfile, "wb" ) as pickle_f:
 pickle.dump( [listwords, embeddings], pickle_f)

# b) Analyse des embedding Glove des légendes

In [59]:
import numpy as np
import _pickle as pickle

outfile = 'Caption_Embeddings.p'

[listwords, embeddings] = pickle.load(open(outfile, "rb"))
print("embeddings:", embeddings.shape)

for i in range(embeddings.shape[0]):
    # L2 NORMALIZATION
    norm = np.linalg.norm(embeddings[i])
    if norm > 0:
        embeddings[i] /= norm


embeddings: (7003, 102)


L’objectif de la normalisation est de ramener chaque embedding à une longueur unitaire, c’est-à-dire à une norme égale à 1. En faisant cela, on ne modifie pas la direction du vecteur, mais on enlève toute information liée à la magnitude ,  ce qui peut faciliter les comparaisons de similarité (par exemple, via la mesure de cosinus) 


In [61]:
from sklearn.cluster import KMeans
# On crée un modèle KMeans avec 10 clusters, init random, et max_iter=1000
kmeans = KMeans(n_clusters=10, init="random", max_iter=1000, random_state=0)
# On applique le modèle aux données
kmeans.fit(embeddings)
clustersID  = kmeans.labels_
clusters = kmeans.cluster_centers_

In [ ]:
from sklearn.manifold import TSNE
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.cm as cm

pointsclusters = 
indclusters = 

for i in range(10):
 norm = np.linalg.norm((clusters[i] - embeddings),axis=1)
 inorms = np.argsort(norm)
 indclusters[i][:] = inorms[:]

 print("Cluster "+str(i)+" ="+listwords[indclusters[i][0]])
 for j in range(1,21):
  print(" mot: "+listwords[indclusters[i][j]])